In [ ]:
import ESN as ESN
import numpy as np
import torch
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

In [ ]:
# This is a demo for DeepESN-LSR with $N_{L}=3$
data = np.loadtxt('./data/mackey_glass_t17_original.txt')
input_data = data[:-84]
label_data = data[84:]
input_data = input_data[:6000]
label_data = label_data[:6000]
total_length = 6000
training_length = 4000
validation_length = 1000
test_length = 1000
washout = 400
Model_input = torch.Tensor(input_data).unsqueeze(1)
Model_target = torch.Tensor(label_data).unsqueeze(1)
# Controlling indeces based on $N_{L}^{3}=4$. You can modify this part to your desired $N_{L}^{3}$ following my writing style.
L2_a_index = list(range(0,total_length,2))
L2_b_index = list(range(1,total_length,2))
L3_c_index = list(range(0,total_length,4))
L3_d_index = list(range(1,total_length,4))
L3_e_index = list(range(2,total_length,4))
L3_f_index = list(range(3,total_length,4))
total_round = 20
reservoir_size = [100]
input_scale = [1]
lr = [1]
spectral_radius =[1]
regularization = [1e-6]
global_validation = np.zeros((len(input_scale),len(lr),len(spectral_radius),len(regularization)))
global_test = np.zeros((len(input_scale),len(lr),len(spectral_radius),len(regularization)))
for h in range(len(reservoir_size)):
    for i in range(len(input_scale)):
        for j in range(len(lr)):
            for k in range(len(spectral_radius)):
                for l in range(len(regularization)):
                    mean_RMSE_validation = []
                    mean_RMSE_test = []
                    mean_NRMSE_validation = []
                    mean_NRMSE_test = []
                    mean_MAPE_validation = []
                    mean_MAPE_test = []
                    mean_MAE_validation = []
                    mean_MAE_test = []
                    mean_R2_validation = []
                    mean_R2_test = []
                    for r in range(total_round):
                        all_index = list(range(0,6000))
                        train_index = list(range(washout,training_length))
                        validation_index = list(range(training_length,training_length+validation_length))
                        test_index = list(range(training_length+validation_length,training_length+validation_length+test_length))

                        esn_layer_1 = ESN.ESN(input = 1,reservoir =reservoir_size[h],sr = spectral_radius[k],density = 0.1,scale_in = input_scale[i],leaking_rate =lr[j],Nepochs=10,eta=1e-3,mu=0,sigma=0.1,threshold=0.1,W_assign='Uniform',Win_assign='Uniform')
                        state_L1_list = esn_layer_1(Model_input,h_0 = None, useIP = False)
                        state_L1 = ESN.state_transform(state_L1_list)
            
                        state_L2_a_input = state_L1[L2_a_index]
                        state_L2_b_input = state_L1[L2_b_index]
                        esn_layer_2_a = ESN.ESN(input = reservoir_size[h],reservoir =reservoir_size[h],sr = spectral_radius[k],density = 0.1,scale_in = input_scale[i],leaking_rate =lr[j],Nepochs=10,eta=1e-3,mu=0,sigma=0.1,threshold=0.1,W_assign='Uniform',Win_assign='Uniform')
                        state_L2_a_list = esn_layer_2_a(state_L2_a_input,h_0=None,useIP=False)
                        state_L2_a = ESN.state_transform(state_L2_a_list)
                        esn_layer_2_b = ESN.ESN(input = reservoir_size[h],reservoir =reservoir_size[h],sr = spectral_radius[k],density = 0.1,scale_in = input_scale[i],leaking_rate =lr[j],Nepochs=10,eta=1e-3,mu=0,sigma=0.1,threshold=0.1,W_assign='Uniform',Win_assign='Uniform')
                        state_L2_b_list = esn_layer_2_b(state_L2_b_input,h_0=None,useIP=False)
                        state_L2_b = ESN.state_transform(state_L2_b_list)
                        state_L2_ab_list = []
                        for q in range(int(total_length/2)):
                            state_L2_ab_list.append(state_L2_a[q].unsqueeze(0))
                            state_L2_ab_list.append(state_L2_b[q].unsqueeze(0))
                        state_L2_ab = ESN.state_transform(state_L2_ab_list)
                        state_L3_c_input = state_L2_ab[L3_c_index]
                        state_L3_d_input = state_L2_ab[L3_d_index]
                        state_L3_e_input = state_L2_ab[L3_e_index]
                        state_L3_f_input = state_L2_ab[L3_f_index]
                        esn_layer_3_c = ESN.ESN(input = reservoir_size[h],reservoir =reservoir_size[h],sr = spectral_radius[k],density = 0.1,scale_in = input_scale[i],leaking_rate =lr[j],Nepochs=10,eta=1e-3,mu=0,sigma=0.1,threshold=0.1,W_assign='Uniform',Win_assign='Uniform')
                        state_L3_c_list = esn_layer_3_c(state_L3_c_input,h_0=None,useIP=False)
                        state_L3_c = ESN.state_transform(state_L3_c_list)
                        esn_layer_3_d = ESN.ESN(input = reservoir_size[h],reservoir =reservoir_size[h],sr = spectral_radius[k],density = 0.1,scale_in = input_scale[i],leaking_rate =lr[j],Nepochs=10,eta=1e-3,mu=0,sigma=0.1,threshold=0.1,W_assign='Uniform',Win_assign='Uniform')
                        state_L3_d_list = esn_layer_3_d(state_L3_d_input,h_0=None,useIP=False)
                        state_L3_d = ESN.state_transform(state_L3_d_list)
                        esn_layer_3_e = ESN.ESN(input = reservoir_size[h],reservoir =reservoir_size[h],sr = spectral_radius[k],density = 0.1,scale_in = input_scale[i],leaking_rate =lr[j],Nepochs=10,eta=1e-3,mu=0,sigma=0.1,threshold=0.1,W_assign='Uniform',Win_assign='Uniform')
                        state_L3_e_list = esn_layer_3_e(state_L3_e_input,h_0=None,useIP=False)
                        state_L3_e = ESN.state_transform(state_L3_e_list)
                        esn_layer_3_f = ESN.ESN(input = reservoir_size[h],reservoir =reservoir_size[h],sr = spectral_radius[k],density = 0.1,scale_in = input_scale[i],leaking_rate =lr[j],Nepochs=10,eta=1e-3,mu=0,sigma=0.1,threshold=0.1,W_assign='Uniform',Win_assign='Uniform')
                        state_L3_f_list = esn_layer_3_f(state_L3_f_input,h_0=None,useIP=False)
                        state_L3_f = ESN.state_transform(state_L3_f_list)
                        state_L3_cdef_list = []
                        for q in range(int(total_length/4)):
                            state_L3_cdef_list.append(state_L3_c[q,:].unsqueeze(0))
                            state_L3_cdef_list.append(state_L3_d[q,:].unsqueeze(0))
                            state_L3_cdef_list.append(state_L3_e[q,:].unsqueeze(0))
                            state_L3_cdef_list.append(state_L3_f[q,:].unsqueeze(0))
                        state_L3_cdef = ESN.state_transform(state_L3_cdef_list)
                        state_concat = (torch.cat((state_L1,state_L2_ab,state_L3_cdef),1)).numpy()

                        train_input = state_concat[train_index]
                        train_target = Model_target[train_index].numpy()
                        validation_input = state_concat[validation_index]
                        validation_target = Model_target[validation_index].numpy()
                        test_input = state_concat[test_index]
                        test_target = Model_target[test_index].numpy()
                        Regressor = Ridge(alpha=regularization[l]).fit(train_input,train_target)
                        pred_train = Regressor.predict(train_input)
                        pred_validation = Regressor.predict(validation_input)
                        pred_test = Regressor.predict(test_input)
                        RMSE_validation = mean_squared_error(validation_target,pred_validation,squared=False)
                        RMSE_test = mean_squared_error(test_target,pred_test,squared=False)
                        NRMSE_validation = RMSE_validation/np.std(validation_target)
                        NRMSE_test = RMSE_test/np.std(test_target)
                        mean_RMSE_validation.append(RMSE_validation)
                        mean_RMSE_test.append(RMSE_test)
                        mean_NRMSE_validation.append(NRMSE_validation)
                        mean_NRMSE_test.append(NRMSE_test)
                    RMSE_val_mean = np.mean(np.array(mean_RMSE_validation))
                    RMSE_val_std = np.std(np.array(mean_RMSE_validation))
                    RMSE_test_mean = np.mean(np.array(mean_RMSE_test))
                    RMSE_test_std = np.std(np.array(mean_RMSE_test))
                    NRMSE_val_mean = np.mean(np.array(mean_NRMSE_validation))
                    NRMSE_val_std = np.std(np.array(mean_NRMSE_validation))
                    NRMSE_test_mean = np.mean(np.array(mean_NRMSE_test))
                    NRMSE_test_std = np.std(np.array(mean_NRMSE_test))
                    #You can check these two matrices to monitor the performances.